# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hack').getOrCreate()

In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer

In [4]:
#Read the data
data = spark.read.csv('/FileStore/tables/hack_data.csv', inferSchema=True, header=True)
data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 20 rows

In [5]:
#Index the location colum
indexer = StringIndexer(inputCol='Location', outputCol='Location_indexed')
data = indexer.fit(data).transform(data)

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-1175643350392374> in <module> 
 2 
 3 indexer = StringIndexer ( inputCol = 'Location' , outputCol = 'Location_indexed' ) 
 ----> 4 data = indexer . fit ( data ) . transform ( data ) 
 5 
 6 data . printSchema

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 127 return self . copy ( params ) . _fit ( dataset ) 
 128 else : 
 --> 129 return self . _fit ( dataset ) 
 130 else : 
 131 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 319 
 320 def _fit ( self , dataset ) : 
 --> 321 java_model = self . _fit_java ( dataset ) 
 322 model = self . _create_model ( java_model ) 
 323 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 316 """
 317 self . _transfer_params_to_java ( ) 
 --> 318 return self . _java_obj . fit ( dataset . _jdf ) 
 319 
 320 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 132 # Hide where the exception came from that shows a non-Pythonic 
 133 # JVM exception message. 
 --> 134 raise_from ( converted ) 
 135 else : 
 136 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 IllegalArgumentException : requirement failed: Output column Location_indexed already exists.

In [6]:
data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- Location_indexed: double (nullable = false)

In [7]:
data.columns

Out[17]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'Location_indexed']

In [8]:
#Assemble the column into a feature column
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed',
 'Location_indexed'], outputCol='features')

In [9]:
data = assembler.transform(data)

In [10]:
data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- Location_indexed: double (nullable = false)
-- features: vector (nullable = true)

In [11]:
#Scale the features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
data_final = scaler.fit(data).transform(data)

## Train the Model and Evaluate

In [13]:
kmeans_2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans_3 = KMeans(featuresCol='scaledFeatures', k=3)

In [14]:
model_2 = kmeans_2.fit(data_final)
model_3 = kmeans_3.fit(data_final)

In [15]:
preds_2 = model_2.transform(data_final)
preds_3 = model_3.transform(data_final)

In [16]:
preds_2.groupBy("prediction").count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [17]:
preds_3.groupBy("prediction").count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 88|
 2| 79|
 0| 167|
+----------+-----+